In [86]:
import bs4
import json
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pyperclip
from selenium.webdriver.common.keys import Keys
from dotenv import load_dotenv
import os
import datetime

- id, password 는 환경변수로 설정


In [84]:
load_dotenv()
id = os.environ.get("MyID")
password = os.environ.get("MyPassword")

In [ ]:
# load the driver
driver = webdriver.Chrome()

# login
driver.get('https://nid.naver.com/nidlogin.login')
time.sleep(1)
pyperclip.copy(id) 
driver.find_element(By.ID,'id').send_keys(Keys.CONTROL, 'v')
pyperclip.copy(password)
driver.find_element(By.ID,'pw').send_keys(Keys.CONTROL, 'v')
driver.find_element(By.ID,'log.login').click()
time.sleep(1)

# move to the cafe
driver.get('https://cafe.naver.com/mbticafe')
# move to the first 사랑방
driver.find_element(By.ID, "menuLink18").click()

contents_list = []
author_list = []
comments_list_with_post = []
comments_list = []
comments_writer_list = []
for page_num in range(1, 4):
    # move to the page
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.XPATH, f'//*[@id="main-area"]/div[6]/a[{page_num}]').click()
    time.sleep(1)
    for i in range(1, 16):
        if i != 1:
            driver.switch_to.frame("cafe_main")
        # enter the post
        content_path = f'//*[@id="main-area"]/div[4]/table/tbody/tr[{i}]/td[1]/div[2]/div/a'
        driver.find_element(By.XPATH, content_path).click()
        time.sleep(1)
        # contents loading
        content = driver.find_elements(By.CSS_SELECTOR, '.se-fs-.se-ff-')
        inner_contents = []
        for i in content:
            inner_contents.append(i.text)
        contents_list.append(inner_contents)
        # author loading
        author = driver.find_element(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[2]/div[2]/div/a/span[2]/strong')
        author_list.append(author.text)
        # comments with post loading
        comments = driver.find_elements(By.XPATH, '//span[@class="text_comment"]')
        inner_contents = []
        if comments == '':
            inner_contents.append('')
        else:
            for i in comments:
                inner_contents.append(i.text)
            comments_list_with_post.append(inner_contents)
        # comments loading
        for i in comments:
            comments_list.append(i.text)
        # comments_writer loading
        comment_writer = driver.find_elements(By.CLASS_NAME, 'comment_nickname')
        for i in comment_writer:
            comments_writer_list.append(i.text)
        
        driver.back()

driver.quit()
print("----------- crawling done -----------")

current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

df_post = pd.DataFrame({'author': author_list, 'contents': contents_list, 'comments': comments_list_with_post})
df_post.to_csv(f'post_{current_time}.csv', index=False, encoding='utf-8-sig')

df_comment = pd.DataFrame({'comments': comments_list, 'comments_writer': comments_writer_list})
df_comment.to_csv(f'comment_{current_time}.csv', index=False, encoding='utf-8-sig')
